In [1]:
import ipywidgets as wgt
import pandas as pd

tdf=pd.read_csv('chess2_rep_moreinfo.tab', sep='\t')

In [2]:
tdf['excov']=pd.to_numeric(tdf['excov'], errors='coerce').fillna(0)
tdf['cdcov']=pd.to_numeric(tdf['cdcov'], errors='coerce').fillna(0)
tdf.shape[0] #number of rows

323827

In [14]:
seltype=wgt.SelectMultiple(
    options=sorted(tdf.type.unique()),
    value=[],
    rows=4,
    description='TYPE',
)
selstatus=wgt.SelectMultiple(
    options=sorted(tdf.status.unique()),
    value=['novel'],
    rows=4,
    description='STATUS',
)
hbox2=wgt.HBox([selstatus, seltype])
display(hbox2)
excov=wgt.IntSlider(value=0, min=0, max=100, description='')
cdcov=wgt.IntSlider(value=0, min=0, max=100, description='')
excount=wgt.BoundedIntText(value=1, min=1, max=10, description='At least',
                          layout=wgt.Layout(width='12em'))
hbox=wgt.HBox([wgt.VBox([wgt.Label(value="Min. exon cov%:"), excov]),
                  wgt.VBox([wgt.Label(value="Min. CDS cov%:"), cdcov]),
              excount, wgt.Label(value="exons")])
display(hbox)

runbtn=wgt.Button(
    description='Query',
    #disabled=False,
    #button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Query',
    #icon='check'
)
rlabel=wgt.Label(value="---", layout=wgt.Layout(border='2px solid red', padding='0 12px'))
rlabelres=wgt.Label(value="---", layout=wgt.Layout(border='2px solid red', padding='0 12px'))
def btnRunClick(b):
    rlabel.value=''
    rdf=tdf
    if len(selstatus.value)==4 or len(selstatus.value)==0:
        rlabel.value='any status'
    else: 
        rlabel.value=' | '.join(selstatus.value)
        rdf=tdf[(tdf['status'].isin(selstatus.value))]
        if len(selstatus.value)==3 and not 'novel' in selstatus.value:
            rlabel.value='known'
        else:
            if len(selstatus.value)>1: rlabel.value='('+rlabel.value+')'
    stype=''
    
    if not(len(seltype.value)==4 or len(seltype.value)==0):
        stype=' | '.join(seltype.value)
        rdf=rdf[(rdf['type'].isin(seltype.value))]
        
    if rlabel.value and stype: 
        rlabel.value+=' &  '
        if len(seltype.value)>1: stype='('+stype+')'
    rlabel.value+=stype
    if excount.value>1:
        rdf=rdf[(rdf['excount']>=excount.value)]
    if excov.value>0:
       rdf=rdf[(rdf['excov']>=excov.value)]    
    if cdcov.value>0:
       rdf=rdf[(rdf['cdcov']>=cdcov.value)]

    rlabelres.value=str(len(rdf))

runbtn.on_click(btnRunClick)

hbx=wgt.HBox(children=[runbtn, rlabel, rlabelres], layout=wgt.Layout(justify_content='center'))
display(hbx)
btnRunClick(True)